In [58]:
import math
import arrow

import ipynb 
import os.path
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from dotenv import load_dotenv
from networkx.algorithms import bipartite
from importlib import reload

from typing import List

load_dotenv(verbose=True)

True

In [59]:
# import functions from py file 

import function
reload(function)
from function import CB_data_cleaning, df_from_api_CB, extract_nodes, extract_data_from_column
from function import nx_dip_graph_from_pandas, plot_bipartite_graph, filter_dict
from function import extract_classes_company_tech, degree_bip, insert_data_classes

In [60]:
# import classes 

import classes
reload(classes)
from classes import Company, Investor, Technology

### Download data from CSV

In [61]:
df_start = pd.read_csv("data/data_cb/organizations.csv")

In [62]:
df_start.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,...,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,158955.0,2007-05-25 13:51:27,2019-06-24 22:19:25,NaN,company,...,206-859-6300,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,6686.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",...,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,1.0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,7793.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,...,877-342-7222,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,136861.0,2007-05-26 03:21:34,2019-06-19 12:17:48,NaN,investor,...,650.482.2500,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,investor,38.0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,47.0,2007-05-26 04:22:15,2021-04-14 23:52:25,"Facebook, Inc.","investor,company",...,NaN,https://www.facebook.com/facebook/,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN


In [63]:
df_start.columns

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'legal_name', 'roles', 'domain', 'homepage_url',
       'country_code', 'state_code', 'region', 'city', 'address',
       'postal_code', 'status', 'short_description', 'category_list',
       'category_groups_list', 'num_funding_rounds', 'total_funding_usd',
       'total_funding', 'total_funding_currency_code', 'founded_on',
       'last_funding_on', 'closed_on', 'employee_count', 'email', 'phone',
       'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'alias1',
       'alias2', 'alias3', 'primary_role', 'num_exits'],
      dtype='object')

### Data Cleaning

for now we decide to use as key the name. it would be better to use the uuid

In [64]:
to_drop = [
    'type',
    'permalink',
    'cb_url',   
    'created_at',
    'domain',
    'address',
    'state_code',
    'updated_at',
    'legal_name',
    'roles',
    'postal_code',
    'homepage_url',
    'num_funding_rounds',
    'total_funding_currency_code',
    'phone',
    'email',
    'num_exits',
    'alias2',
    'alias3',
    'num_exits',
    'logo_url',
    'alias1',
    'last_funding_on',
    'twitter_url',
    'facebook_url'
]

to_rename = { 'category_groups_list': 'category_groups' }

drop_if_nan = [
    'category_groups',
    'rank'
]

to_check_double = {}

sort_by = "rank"

In [65]:
df = CB_data_cleaning(df_start, to_drop, to_rename, to_check_double, drop_if_nan, sort_by)

In [66]:
df = df[:10]

In [67]:
df.head()

,uuid,name,rank,country_code,region,city,status,short_description,category_list,category_groups,total_funding_usd,total_funding,founded_on,closed_on,employee_count,linkedin_url,primary_role
1178,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,Ant Group,1.0,CHN,Zhejiang,Hangzhou,operating,Ant Group strives to enable all consumers and ...,"Banking,Financial Services,FinTech,Payments","Financial Services,Lending and Investments,Pay...",2.200000e+10,2.200000e+10,2014-10-01,NaN,5001-10000,https://www.linkedin.com/company/antgroup/,company
4042,022417b5-4980-6c54-0f3c-6736bbbb1a5e,Spotify,2.0,SWE,Stockholms Lan,Stockholm,ipo,Spotify is a commercial music streaming servic...,"Audio,Cloud Computing,Music,Music Streaming,Vi...","Content and Publishing,Internet Services,Media...",2.085425e+09,2.085425e+09,2006-04-23,NaN,5001-10000,http://www.linkedin.com/company/spotify,company
349,468bef9f-2f50-590e-6e78-62e3adb05aa1,Citi,3.0,USA,New York,New York,ipo,Citigroup is a diversified financial services ...,"Banking,Credit Cards,Financial Services,Wealth...","Financial Services,Lending and Investments,Pay...",8.700000e+09,8.700000e+09,1998-10-08,NaN,10000+,https://www.linkedin.com/company/citi,investor
211260,a40d0a1f-f32c-a1e9-1bbd-a10bb0eca2e7,Deliveroo,4.0,GBR,England,London,ipo,Deliveroo is an online food delivery service t...,"Delivery,Food and Beverage,Food Delivery,Same ...","Administrative Services,Food and Beverage,Tran...",1.712683e+09,1.712683e+09,2012-01-01,NaN,5001-10000,https://www.linkedin.com/company/deliveroo,company
621119,00daca16-8311-454b-84e0-24a40d16be9c,Antler,5.0,SGP,Central Region,Singapore,operating,Antler is a global early-stage venture capital...,Venture Capital,"Financial Services,Lending and Investments",7.800000e+07,7.800000e+07,2017-01-01,NaN,101-250,https://www.linkedin.com/company/antlerglobal/,investor


In [98]:
df.columns

Index(['uuid', 'name', 'rank', 'country_code', 'region', 'city', 'status',
       'short_description', 'category_list', 'category_groups',
       'total_funding_usd', 'total_funding', 'founded_on', 'closed_on',
       'employee_count', 'linkedin_url', 'primary_role'],
      dtype='object')

In [99]:
df_places = df[['country_code', 'region', 'city']]
df_places.head()

,country_code,region,city
1178,CHN,Zhejiang,Hangzhou
4042,SWE,Stockholms Lan,Stockholm
349,USA,New York,New York
211260,GBR,England,London
621119,SGP,Central Region,Singapore


In [111]:
!pip install haversine

In [112]:
import haversine as hs

In [113]:
loc1=(28.426846,77.088834)
loc2=(28.394231,77.050308)
hs.haversine(loc1,loc2)


5.229712941541709

In [100]:
from geopy.geocoders import Nominatim
import geopandas
import geocoder

In [101]:
geolocator = Nominatim(user_agent='myapplication')

In [102]:
import geocoder

In [108]:
def extract_coordinates_location(location_company):
    
    str_place = location_company['city'] + ', ' + location_company['region'] #+ ', ' +  row['country_code']

    address = geolocator.geocode(str_place) # coversion to conventional address (valid for the next command)
    
    lat = address.latitude
    lon = address.longitude
    
    return address, lat, lon 

In [110]:
extract_coordinates_location(row)

New York, United States


(40.7127281, -74.0060152)

In [107]:
for index, row in df_places.iterrows():
    
    str_place = row['city'] + ', ' + row['region'] #+ ', ' +  row['country_code']

    location = geolocator.geocode(str_place) # coversion to conventional address (valid for the next command)
    print(location)
    
    lat = location.latitude
    lon = location.longitude
    
    # update the class with latitude and longitude
    
    
    print(lat)

杭州市, 四季青街道, 浙江省, 中国
30.2489634
Stockholm, Stockholms kommun, Stockholms län, 111 29, Sverige
59.3251172
New York, United States
40.7127281
London, Greater London, England, United Kingdom
51.5073219
Ministry of Foreign Affairs, Singapore, Camp Road, Tanglin, Singapore, Central, 248819, Singapore
1.30420465
San Francisco, San Francisco City and County, San Francisco, California, United States
37.7790262
San Francisco, San Francisco City and County, San Francisco, California, United States
37.7790262
杭州市, 四季青街道, 浙江省, 中国
30.2489634
Los Gatos, Santa Clara County, California, United States
37.226611
New York, United States
40.7127281


In [104]:
address.latitude

40.7127281

In [105]:
lat = location.latitude
lon = location.longitude

AttributeError: 'NoneType' object has no attribute 'latitude'